![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

## Data Gathering

In [479]:
# Importing Pandas and NumPy libraries.
import pandas as pd
import numpy as np

# Reading data from 'bank_marketing.csv' into a DataFrame.
df = pd.read_csv('bank_marketing.csv')
print(df.head())

   client_id  age        job  ... euribor3m nr_employed   y
0          0   56  housemaid  ...     4.857      5191.0  no
1          1   57   services  ...     4.857      5191.0  no
2          2   37   services  ...     4.857      5191.0  no
3          3   40     admin.  ...     4.857      5191.0  no
4          4   56   services  ...     4.857      5191.0  no

[5 rows x 22 columns]


### Creating needed Dataframes

In [480]:
print(df.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')


In [481]:
# Creating a new DataFrame 'client' by selecting columns 0 through 7 from the df dataframe
client = df.iloc[:, 0:8]
client.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [482]:
# Selecting the columns for the campaign dataframe
cam_cols = ['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome','y', 'day', 'month']

# Creating the campaign dataframe by selecting the specified columns from the df dataframe
campaign = df[cam_cols]

# Displaying the first few rows of the campaign dataframe
campaign.head()

,client_id,campaign,duration,pdays,previous,poutcome,y,day,month
0,0,1,261,999,0,nonexistent,no,13,may
1,1,1,149,999,0,nonexistent,no,19,may
2,2,1,226,999,0,nonexistent,no,23,may
3,3,1,151,999,0,nonexistent,no,27,may
4,4,1,307,999,0,nonexistent,no,3,may


In [483]:
# Selecting the columns for the economics dataframe
econ_cols = ['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']

# Creating the economics dataframe by selecting the specified columns from the df dataframe
economics = df[econ_cols]

# Displaying the first few rows of the economics dataframe
economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


## Data Cleaning

In [484]:
print(client.dtypes)

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default    object
housing           object
loan              object
dtype: object


In [485]:
# Creating dictionaries to specify the column name replacements for the dataframes
clt_rep = {'client_id' : 'id'}
cam_rep = {'duration' : 'contact_duration', 'previous' : 'previous_campaign_contacts',
          'y' : 'campaign_outcome', 'poutcome' : 'previous_outcome', 
           'campaign' : 'number_contacts'}
eco_rep = {'euribor3m' : 'euribor_three_months', 'nr_employed' : 'number_employed'}


# Renaming the columns of the `client` dataframe using the `clt_rep` dictionary
client = client.rename(columns = clt_rep)

# Renaming the columns of the `campaign` dataframe using the `cam_rep` dictionary
campaign = campaign.rename(columns = cam_rep)

# Renaming the columns of the `economics` dataframe using the `eco_rep` dictionary
economics = economics.rename(columns = eco_rep)

# Printing the column names of the `client`, `campaign`, and `economics` dataframes
print(client.columns, '\n', campaign.columns, '\n', economics.columns)

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object') 
 Index(['client_id', 'number_contacts', 'contact_duration', 'pdays',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'day', 'month'],
      dtype='object') 
 Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


In [486]:
client['education'].unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'unknown', 'university.degree',
       'illiterate'], dtype=object)

In [487]:
# Cleaning education column
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.NAN)

# Test
client['education'].unique()

array(['basic_4y', 'high_school', 'basic_6y', 'basic_9y',
       'professional_course', nan, 'university_degree', 'illiterate'],
      dtype=object)

In [488]:
client['job'].unique()

array(['housemaid', 'services', 'admin.', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [489]:
# Cleaning job column
client['job'] = client['job'].str.replace('.', '')

# Test
client['job'].unique()

array(['housemaid', 'services', 'admin', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [490]:
print(campaign['previous_outcome'].unique())
print(campaign['campaign_outcome'].unique())

['nonexistent' 'failure' 'success']
['no' 'yes']


In [491]:
# Defining a dictionary to map values to binary 1 and 0
bools = {'success' : 1, 'yes' : 1, 'failure' : 0, 'no' : 0}

# Converting values in 'previous_outcome' and 'campaign_outcome' columns to binary using .map()
campaign['previous_outcome'] = campaign['previous_outcome'].map(bools)
campaign['campaign_outcome'] = campaign['campaign_outcome'].map(bools)

# Test
print(campaign['previous_outcome'].unique())
print(campaign['campaign_outcome'].unique())

[nan  0.  1.]
[0 1]


In [492]:
# Add campaign_id column
campaign['campaign_id'] =1

# Test 
campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,day,month,campaign_id
0,0,1,261,999,0,NaN,0,13,may,1
1,1,1,149,999,0,NaN,0,19,may,1
2,2,1,226,999,0,NaN,0,23,may,1
3,3,1,151,999,0,NaN,0,27,may,1
4,4,1,307,999,0,NaN,0,3,may,1


In [493]:
# - Converting 'day' to a string, capitalizing 'month' values, and setting 'year' to '2022'
campaign['day'] = campaign['day'].astype('str')
campaign['month'] = campaign['month'].str.capitalize()
campaign['year'] = '2022'

# Addding last_contact_date column
campaign['last_contact_date'] = campaign['year'] + '-' + campaign['month'] + '-' + campaign['day']

# Converting to datetime
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], 
                                               format="%Y-%b-%d")
# Test
campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,day,month,campaign_id,year,last_contact_date
0,0,1,261,999,0,NaN,0,13,May,1,2022,2022-05-13
1,1,1,149,999,0,NaN,0,19,May,1,2022,2022-05-19
2,2,1,226,999,0,NaN,0,23,May,1,2022,2022-05-23
3,3,1,151,999,0,NaN,0,27,May,1,2022,2022-05-27
4,4,1,307,999,0,NaN,0,3,May,1,2022,2022-05-03


In [494]:
# Dropping unneccessary columns
campaign = campaign.drop(['day', 'month', 'year'], axis = 1)
campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-05-13
1,1,1,149,999,0,NaN,0,1,2022-05-19
2,2,1,226,999,0,NaN,0,1,2022-05-23
3,3,1,151,999,0,NaN,0,1,2022-05-27
4,4,1,307,999,0,NaN,0,1,2022-05-03


In [495]:
# Saving dataframes to individual csv files
client.to_csv('client.csv', index = False)
campaign.to_csv('campaign.csv', index = False)
economics.to_csv('economics.csv', index = False)

## Designing Database

In [496]:
# Creating client table for database_design
client_table = """
CREATE TABLE client (
    id serial PRIMARY KEY,
    age integer,
    job text,
    marital text,
    education text,
    credit_default boolean,
    housing boolean,
    loan boolean
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [497]:
# Creating campaign table for database_design
campaign_table = """
CREATE TABLE campaign (
    campaign_id serial PRIMARY KEY,
    client_id serial REFERENCES client (id),
    number_contacts integer,
    contact_duration integer,
    pdays integer,
    previous_campaign_contacts integer,
    previous_outcome boolean,
    campaign_outcome boolean,
    last_contact_date date
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [498]:
# Creating economics table for database_design
economics_table = """
CREATE TABLE economics (
    client_id serial REFERENCES client (id),
    emp_var_rate float,
    cons_price_idx float,
    euribor_three_months float,
    number_employed float
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""